In [16]:
import os
import numpy as np
import pandas as pd
import joblib
import json
import tensorflow as tf
import gc

In [17]:
tuning_path = '../tunings/TrigL2_20180125_v8'
tuning_file = "ElectronRingerLooseTriggerConfig.json"
with open(os.path.join(tuning_path,tuning_file)) as f:
    conf = json.load(f)

In [3]:
#conf

In [32]:
def open_zee(path= ""):
    '''
    This function will get a .npz file and transform into a pandas DataFrame. 
    The .npz has three types of data: float, int and bool this function will concatenate these features and reorder them.

    Arguments:
    path (str) - the full path to .npz file
    '''
    # open the file
    d = dict(np.load(path, allow_pickle=True))
    print(d.keys())
    df = pd.DataFrame(d['data'],columns=d["features"])
    
    df['target'] = d['target']
    df['et_bin'] = d['etBinIdx']
    df['eta_bin'] = d['etaBinIdx']
    return df

In [33]:
# my pc path
#data_path = '/media/natmourajr/Backup/Work/CERN/Qualify/cern_data/Zee_boosted/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins_et0_eta0.npz'
# LPS path
data_path = '/home/natmourajr/Workspace/CERN/Qualify/data/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et4_eta0.npz'
df = open_zee(data_path)

dict_keys(['features', 'etBins', 'etaBins', 'etBinIdx', 'etaBinIdx', 'data', 'target'])


In [34]:
df.head()

,avgmu,L2Calo_ring_0,L2Calo_ring_1,L2Calo_ring_2,L2Calo_ring_3,L2Calo_ring_4,L2Calo_ring_5,L2Calo_ring_6,L2Calo_ring_7,L2Calo_ring_8,...,el_lhmedium,el_lhloose,el_lhvloose,T0HLTElectronT2CaloTight,T0HLTElectronT2CaloMedium,T0HLTElectronT2CaloLoose,T0HLTElectronT2CaloVLoose,target,et_bin,eta_bin
0,31.459999,946.455200,297.285675,-162.731384,238.094727,-204.648712,112.230659,29.744968,43.871647,4150.571289,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,4,0
1,31.420000,1155.483643,231.537582,841.838196,-32.028019,317.318054,412.808563,19.779219,-101.355553,12582.793945,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,4,0
2,31.459999,2306.196045,123.935341,-92.987045,-264.076416,172.244553,-83.377022,109.801102,-15.553493,8039.985840,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,4,0
3,31.459999,1661.695190,311.053619,-193.597885,-127.421242,386.362854,156.423721,182.063248,-88.269615,6214.927246,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,4,0
4,31.459999,716.282349,572.603027,24.837866,-122.357193,-38.153748,-149.753250,205.552399,-74.376228,4822.833984,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,4,0


In [47]:
def add_tuning_decision_zee(df, tuning_path, config_dict):
        #find et and eta bin
        idx = (df['et_bin']+df['eta_bin']).unique()[0]
        #print(idx)

        m_path = tuning_path
        m_model_path = os.path.join(m_path,config_dict["Model__path"][idx].replace('.onnx',''))
        with open(m_model_path + '.json', 'r') as f:
                sequential = json.load(f) 
        model = tf.keras.models.model_from_json(json.dumps(sequential, separators=(',', ':')))
        model.load_weights(m_model_path + '.h5')
        slope, offset = config_dict["Threshold__slope"][idx],config_dict["Threshold__offset"][idx]

        # ring list
        rings = ['L2Calo_ring_%i' %(iring) for iring in range(100)] 

        def norm1( data ): 
                norms = np.abs( data.sum(axis=1) )
                norms[norms==0] = 1
                return data/norms[:,None] 
        df['nn_output_%s_%s'%(config_dict['__operation__'].lower(),config_dict['__version__'])] = model.predict(norm1(df[rings].values)) 
        df['thr_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = df['avgmu']*slope + offset
        df['nn_decision_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = 0
        df.loc[df['nn_output_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] > df['thr_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])],'nn_decision_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = 1 


In [46]:
# ring list
rings = ['L2Calo_ring_%i' %(iring) for iring in range(100)] 

def norm1( data ): 
                norms = np.abs( data.sum(axis=1) )
                norms[norms==0] = 1
                return data/norms[:,None] 
norm1(df[rings]).values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  import sys


array([[ 1.9066030e-02,  5.9887227e-03, -3.2781705e-03, ...,
         9.5888981e-06,  9.3012309e-04, -1.2849123e-03],
       [ 9.5097004e-03,  1.9055683e-03,  6.9283792e-03, ...,
         5.7096506e-05,  1.5111542e-03,  2.3599887e-04],
       [ 4.3169573e-02,  2.3199397e-03, -1.7406201e-03, ...,
         7.7918265e-04,  7.6186750e-04,  6.2334619e-04],
       ...,
       [ 1.3338522e-02,  9.1572236e-03,  4.3471903e-03, ...,
         9.3081193e-03, -7.7347824e-05,  3.9268893e-05],
       [ 4.3570022e-03,  1.0661377e-02,  6.5613580e-03, ...,
         3.5762454e-03,  4.0346073e-04,  2.1633768e-04],
       [ 2.7849372e-02,  4.1435979e-02,  7.9749068e-03, ...,
         2.6624401e-03,  1.1911067e-03,  2.4349712e-05]], dtype=float32)

In [49]:
tuning_path = '../tunings/TrigL2_20180125_v8'
add_tuning_decision_zee(df=df, tuning_path=tuning_path, config_dict=conf)

4


In [51]:
df.head()

,avgmu,L2Calo_ring_0,L2Calo_ring_1,L2Calo_ring_2,L2Calo_ring_3,L2Calo_ring_4,L2Calo_ring_5,L2Calo_ring_6,L2Calo_ring_7,L2Calo_ring_8,...,T0HLTElectronT2CaloTight,T0HLTElectronT2CaloMedium,T0HLTElectronT2CaloLoose,T0HLTElectronT2CaloVLoose,target,et_bin,eta_bin,nn_output_loose_v8,thr_loose_v8,nn_decision_loose_v8
0,31.459999,946.455200,297.285675,-162.731384,238.094727,-204.648712,112.230659,29.744968,43.871647,4150.571289,...,1.0,1.0,1.0,1.0,1,4,0,-1.728537,0.013493,0
1,31.420000,1155.483643,231.537582,841.838196,-32.028019,317.318054,412.808563,19.779219,-101.355553,12582.793945,...,1.0,1.0,1.0,1.0,1,4,0,-1.040594,0.013974,0
2,31.459999,2306.196045,123.935341,-92.987045,-264.076416,172.244553,-83.377022,109.801102,-15.553493,8039.985840,...,1.0,1.0,1.0,1.0,1,4,0,-1.260863,0.013493,0
3,31.459999,1661.695190,311.053619,-193.597885,-127.421242,386.362854,156.423721,182.063248,-88.269615,6214.927246,...,1.0,1.0,1.0,1.0,1,4,0,-1.103993,0.013493,0
4,31.459999,716.282349,572.603027,24.837866,-122.357193,-38.153748,-149.753250,205.552399,-74.376228,4822.833984,...,1.0,1.0,1.0,1.0,1,4,0,-1.382297,0.013493,0
